In [1]:
import sqlite3
import pandas as pd
from datetime import date
def calculate_age(born):              #считаем возраст клиента по дате рождения
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
def city(a):                          #определяем город по первым буквам кода
    if a[0:3] == 'МСК':
        return 'Москва'
    elif a[0:3] == 'СПБ':
        return 'Санкт Петербург'
    else:
        return 'X'
def sex(a):                           #определяем пол клиента по последней букве отчества
    if a.replace(' ', '')[-1] == 'ч':
        return 'Мужчина'
    elif a.replace(' ', '')[-1] == 'а':
        return 'Женщина'
    else:
        return 'X'
def kategory(a):                      #определяем категорию по коду до дефиса
    if a.split('-')[0] == 'Д':
        return 'Диван'
    elif a.split('-')[0] == 'К':
        return 'Кресло'
    elif a.split('-')[0] == 'ДУ':
        return 'Диван угловой'
    else:
        return 'X'
def collection(a):                    #определяем название коллекции по первой букве после дефиса
    if a.split('-')[1][0] == 'Д':
        return 'Динс'
    elif a.split('-')[1][0] == 'С':
        return 'Спейс'
    elif a.split('-')[1][0] == 'Б':
        return 'Берг'
    elif a.split('-')[1][0] == 'М':
        return 'Магно'
    elif a.split('-')[1][0] == 'В':
        return 'Верона'
    else:
        return 'X'

In [2]:
path = 'E:\\Скилбокс\\Итоговый проект\\'        #прописыаем путь к папке с данными
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select * from cancel_reasons'
    cancel_reasons = pd.read_sql(s, con)
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select id, fio, birth_date from customers'
    customers = pd.read_sql(s, con)
customers['sex'] = customers.fio.apply(sex)
customers.birth_date = pd.to_datetime(customers.birth_date, format="%Y-%m-%d %H:%M:%S")
customers['age'] = customers.birth_date.apply(calculate_age)
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select * from order_rows'
    order_rows = pd.read_sql(s, con)
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select * from order_sources'
    order_sources = pd.read_sql(s, con)
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select * from orders'
    orders = pd.read_sql(s, con)
orders['city'] = orders.order_num.apply(city)
orders.order_date = pd.to_datetime(orders.order_date, format="%Y-%m-%d %H:%M:%S")
orders['counted'] = orders.query('state == "Доставлен"').sort_values(['order_date', 'order_num'])\
                        .groupby(['customer_id']).cumcount()+1         #считаем порядовый номер заказа для каждого клиента
orders.counted.fillna(0, inplace=True)
orders['discount'] = (orders.query('counted >= 6').counted - 6)*500    #вычисляем сумму дисконта начина с 7-го заказа
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select * from sku'
    sku = pd.read_sql(s, con)
sku['collection_name'] = sku.sku_id.apply(collection)
sku['kategory'] = sku.sku_id.apply(kategory)
with sqlite3.connect(path + 'mebel.db') as con:
    s = 'select * from suppliers'
    suppliers = pd.read_sql(s, con)
ozon_nov_sale = pd.read_excel(path + 'Ozon-Nov-Sale.xlsx') 

### Объединяем таблицу orders с вспомагательными таблицами

In [3]:
orders = orders.merge(cancel_reasons, how='left', left_on='cancel_reason_id', right_on='id')              
orders.id.fillna(0, inplace=True)
orders.pop('id')
orders.cancel_reason_id.fillna(value=0, inplace=True)
orders.pop('cancel_reason_id')
orders = orders.merge(order_sources, how='left', left_on='source_id', right_on='id')
orders.source_id.fillna(0, inplace=True)
orders.pop('source_id')
orders.id.fillna(0, inplace=True)
orders.pop('id')
orders = orders.merge(ozon_nov_sale, how='left', left_on='order_num', right_on='Номера заказов')
orders['Номера заказов'].fillna(value=0, inplace=True)
orders.pop('Номера заказов')
orders.counted = orders.counted.astype(int)

In [4]:
sku = sku.merge(suppliers, how='left', left_on='id_supplier', right_on='id')              
sku.pop('id_supplier')
sku.pop('id')
order_rows = order_rows.merge(sku, how='left', on='sku_id') #объединяем таблицу order_rows с вспомагательными таблицами
orders = orders\
        .merge(order_rows.groupby('order_num', as_index=False).agg({'price':'sum'})\
        .rename(columns={'price':'summa'}), how='left', on='order_num')     #вычисляем стоимость каждого заказа         
orders['real_sum'] = orders.query('state == "Доставлен"')['summa']          #оставляем для расчета только доставленные заказы
orders['discount2'] = orders.query('counted == 6').counted - 6 + 250        #вычисляем дополнительный дисконт за 6 заказ
orders.discount2.fillna(0, inplace=True)
orders.discount.fillna(0, inplace=True)
orders.real_sum.fillna(0, inplace=True)
orders['Скидка'].fillna(0, inplace=True)
#считаем стоимость выполненного заказа с учетом купонов и скидок
orders['result'] = round((orders.real_sum - orders.discount)*(1 - orders['Скидка']),2) 
orders.head()

,order_num,order_date,customer_id,state,city,counted,discount,reason,source_name,Скидка,summa,real_sum,discount2,result
0,МСК-1003,2021-01-01,299,Отменен,Москва,0,0.0,Некачественный товар,Мебельград,0.0,27980,0.0,0.0,0.0
1,СПБ-1002,2021-01-01,148,Доставлен,Санкт Петербург,1,0.0,NaN,Три кита,0.0,27490,27490.0,0.0,27490.0
2,МСК-1001,2021-01-01,155,Доставлен,Москва,1,0.0,NaN,Гранд,0.0,27490,27490.0,0.0,27490.0
3,СПБ-1000,2021-01-01,340,Доставлен,Санкт Петербург,1,0.0,NaN,Мебельград,0.0,25980,25980.0,0.0,25980.0
4,МСК-1005,2021-01-02,160,Отменен,Москва,0,0.0,Нет товара на складе,Ozon.ru,0.0,13990,0.0,0.0,0.0


### Дополняем таблицу customers счетчиком заказов

In [5]:
customers = customers\
          .merge(orders.query('state == "Доставлен"').groupby('customer_id', as_index=False)\
          .agg({'order_num':'count'}), how='left', left_on='id', right_on='customer_id')\
          .rename(columns={'order_num':'orders'})
customers.orders.fillna(0, inplace=True)
customers.orders = customers.orders.astype(int)
customers.customer_id.fillna(0, inplace=True)
customers.pop("customer_id")
customers.head()

,id,fio,birth_date,sex,age,orders
0,1,Никифоров Василий Михайлович,1994-02-01,Мужчина,28,5
1,2,Лебедева Анна Артёмовна,1972-06-22,Женщина,50,4
2,3,Безрукова Анастасия Михайловна,1984-10-18,Женщина,38,4
3,4,Новиков Никита Дмитриевич,1979-08-10,Мужчина,43,3
4,5,Васильева Екатерина Ивановна,1992-03-31,Женщина,30,4
